In [1]:
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp

import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

sc.settings.verbosity = 3
sc.logging.print_versions()

from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import scrublet as scr

-----
anndata     0.7.5
scanpy      1.8.1
sinfo       0.3.1
-----
PIL                 6.2.0
anndata             0.7.5
anndata2ri          1.0.2
backcall            0.1.0
bottleneck          1.2.1
cairo               1.19.1
cffi                1.12.3
cloudpickle         1.2.2
colorama            0.4.1
cycler              0.10.0
cython_runtime      NA
cytoolz             0.10.0
dask                2.5.2
dateutil            2.8.0
decorator           4.4.0
get_version         2.1
google              NA
h5py                2.10.0
igraph              0.8.2
ipykernel           5.1.2
ipython_genutils    0.2.0
ipywidgets          7.5.1
jedi                0.15.1
jinja2              2.10.3
joblib              0.13.2
kiwisolver          1.1.0
leidenalg           0.7.0
llvmlite            0.37.0
louvain             0.7.0
markupsafe          1.1.1
matplotlib          3.3.4
more_itertools      NA
mpl_toolkits        NA
natsort             7.0.0
numba               0.54.0
numexpr             2.7.0
nu

/home/sennis/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# read in datasets
ennis = sc.read('../data/nuig/ennis.h5ad')
ennis.obs = ennis.obs[['dataset', 'sample', 'donor', 'timepoint', 'celltype']]
ennis

AnnData object with n_obs × n_vars = 121314 × 36601
    obs: 'dataset', 'sample', 'donor', 'timepoint', 'celltype'

In [23]:
# read in datasets
vang = sc.read('../data/van_Galen/van_Galen.h5ad')
vang.obs['timepoint'] = vang.obs['sample'].str.extract(r'(D[0-9]+$)')
tp = np.array(vang.obs['timepoint'])
tp[vang.obs['donor_status'] == 'healthy'] = 'Healthy'
vang.obs['timepoint'] = tp
vang.obs = vang.obs[['dataset', 'sample', 'donor', 'timepoint', 'celltype']]
vang.X = vang.layers['counts'].copy()
del vang.uns, vang.layers['counts'], vang.obsm, vang.obsp, vang.varm, vang.var['highly_variable'], vang.var['means'], vang.var['dispersions'], vang.var['dispersions_norm']
vang

AnnData object with n_obs × n_vars = 38410 × 27899
    obs: 'dataset', 'sample', 'donor', 'timepoint', 'celltype'

In [70]:
# read in datasets
pet = sc.read('../data/petti/petti.h5ad')
pet.obs['dataset'] = 'petti'
pet.obs['donor'] = pet.obs['sample'].str.extract(r'([0-9]+)')
pet.obs = pet.obs[['dataset', 'sample', 'donor', 'timepoint', 'celltype']]
pet.X = pet.layers['counts'].copy()
del pet.uns, pet.layers['counts'], pet.obsm, pet.obsp, pet.varm, pet.var['highly_variable'], pet.var['means'], pet.var['dispersions'], pet.var['dispersions_norm']
pet

AnnData object with n_obs × n_vars = 103612 × 22164
    obs: 'dataset', 'sample', 'donor', 'timepoint', 'celltype'

In [72]:
# read in datatsets
gran = sc.read('../data/granja/granja.h5ad')
gran.obs['dataset'] = 'granja'
gran.obs['sample'] = gran.obs['Group']
gran.obs['donor'] = gran.obs['sample'].str.extract(r'(_D[1-3])')[0].str.extract(r'(D[1-3])')
gran.obs['timepoint'] = 'Healthy'
gran.obs = gran.obs[['dataset', 'sample', 'donor', 'timepoint', 'celltype']]
gran.X = gran.layers['counts'].copy()
del gran.uns, gran.layers['counts']
gran

AnnData object with n_obs × n_vars = 20558 × 20287
    obs: 'dataset', 'sample', 'donor', 'timepoint', 'celltype'

In [73]:
# read in datasets
oet = sc.read('../data/oetjen/oetjen.h5ad')
oet.obs['dataset'] = 'oetjen'
oet.obs['donor'] = oet.obs['patient']
oet.obs['timepoint'] = 'Healthy'
oet.obs = oet.obs[['dataset', 'sample', 'donor', 'timepoint', 'celltype']]
oet.X = oet.layers['counts'].copy()
del oet.layers['counts'], oet.uns, oet.obsm, oet.varm, oet.obsp, oet.var['highly_variable'], oet.var['means'], oet.var['dispersions'], oet.var['dispersions_norm']
oet

AnnData object with n_obs × n_vars = 73050 × 33694
    obs: 'dataset', 'sample', 'donor', 'timepoint', 'celltype'

In [74]:
# merge adatas
datasets = [gran, oet, vang, pet, ennis]
adata_merge = datasets[0].concatenate(datasets[1:], join='inner')
print(adata_merge.shape)

/home/sennis/anaconda3/lib/python3.7/site-packages/anndata/_core/merge.py:894: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  UserWarning,


(356944, 19109)


In [77]:
# filter out genes expressed in less than 3 cells
sc.pp.filter_genes(adata_merge, min_cells = 3)
print(adata_merge.shape)

(356944, 18093)


In [78]:
# write out results
adata_merge.write('../data/bone_marrow_merge.h5ad')

... storing 'dataset' as categorical
... storing 'donor' as categorical
... storing 'timepoint' as categorical


To normalise the data, I'm going to use scran's function for normalisation in clusters.

In [80]:
# normalise inner-join data
#Perform a clustering for scran normalisation in clusters
adata_pp = adata_merge.copy()
sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e4)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=15)
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added='groups', resolution=0.5)

normalizing by total count per cell
    finished (0:00:05): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)
computing PCA
    with n_comps=15
    finished (0:01:38)
computing neighbors
    using 'X_pca' with n_pcs = 15


/home/sennis/anaconda3/lib/python3.7/site-packages/numba/core/typed_passes.py:332: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../../home/sennis/anaconda3/lib/python3.7/site-packages/umap/rp_tree.py", line 135:
@numba.njit(fastmath=True, nogil=True, parallel=True)
def euclidean_random_projection_split(data, indices, rng_state):
^

  state.func_ir.loc))
/home/sennis/anaconda3/lib/python3.7/site-packages/umap/nndescent.py:92: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../../home/sennis/anaconda3/

    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:01:10)
running Leiden clustering
    finished: found 32 clusters and added
    'groups', the cluster labels (adata.obs, categorical) (0:04:04)


In [81]:
#Preprocess variables for scran normalisation
input_groups = adata_pp.obs['groups']
data_mat = adata_merge.X.T

In [82]:
%%R -i data_mat -i input_groups -o size_factors
library(scran)
size_factors = computeSumFactors(data_mat, clusters=input_groups, min.mean=0.1)

R[write to console]: Loading required package: SingleCellExperiment

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: GenomicRanges

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following object is masked from ‘package:Matrix’:

    which


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame

In [83]:
#Delete adata_pp
del adata_pp
#Keep the count data in a counts layer
adata_merge.layers["counts"] = adata_merge.X.copy()
#Normalise adata 
adata_merge.obs['size_factors'] = size_factors
adata_merge.write('../data/bone_marrow_merge.h5ad')

In [6]:
adata_merge.X /= adata_merge.obs['size_factors'].values[:,None]
adata_merge.X = sp.sparse.csr_matrix(adata_merge.X)
sc.pp.log1p(adata_merge)
## write out file with normalised data
adata_merge.write('../data/bone_marrow_merge.h5ad')

## Find doublets with scrublet

In [7]:
adata_merge.obs = adata_merge.obs.astype({"sample":'category'})
samples = adata_merge.obs['sample'].cat.categories.tolist()
bcs=[]
dbls=[]
scrs=[]
for i in samples:
    sub = adata_merge[adata_merge.obs['sample'] == i].copy()
    counts_matrix = sp.sparse.csc_matrix(sub.layers['counts'])
    genes = np.array(sub.var_names.values)
    scrub = scr.Scrublet(counts_matrix)
    doublet_scores, predicted_doublets = scrub.scrub_doublets()
    bcs.append(sub.obs_names.tolist())
    dbls.append(predicted_doublets.tolist())
    scrs.append(doublet_scores.tolist())

Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.52
Detected doublet rate = 0.2%
Estimated detectable doublet fraction = 10.8%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 2.2%
Elapsed time: 3.3 seconds
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.49
Detected doublet rate = 0.4%
Estimated detectable doublet fraction = 17.2%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 2.5%
Elapsed time: 2.7 seconds
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.42
Detected doublet rate = 1.8%
Estimated detectable doublet fraction = 25.1%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 7.2%
Elapsed time: 7.4 seconds
Preprocessing...
Simulating double

Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.47
Detected doublet rate = 0.3%
Estimated detectable doublet fraction = 8.9%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 3.1%
Elapsed time: 0.8 seconds
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.80
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.2%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 0.0%
Elapsed time: 15.4 seconds
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.52
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 19.0%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 0.0%
Elapsed time: 1.1 seconds
Preprocessing...
Simulating doublets...
Embedding tr

Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.64
Detected doublet rate = 0.5%
Estimated detectable doublet fraction = 2.1%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 22.0%
Elapsed time: 1.8 seconds
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.54
Detected doublet rate = 0.1%
Estimated detectable doublet fraction = 2.6%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 3.2%
Elapsed time: 0.9 seconds
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.57
Detected doublet rate = 0.3%
Estimated detectable doublet fraction = 5.6%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 4.9%
Elapsed time: 1.1 seconds
Preprocessing...
Simulating doublets

Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.49
Detected doublet rate = 1.1%
Estimated detectable doublet fraction = 28.5%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 4.0%
Elapsed time: 4.5 seconds
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.43
Detected doublet rate = 1.9%
Estimated detectable doublet fraction = 37.3%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 5.0%
Elapsed time: 5.8 seconds
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.49
Detected doublet rate = 0.7%
Estimated detectable doublet fraction = 29.8%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 2.5%
Elapsed time: 2.1 seconds
Preprocessing...
Simulating doublets...
Embedding t

Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.50
Detected doublet rate = 1.2%
Estimated detectable doublet fraction = 20.0%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 6.0%
Elapsed time: 2.7 seconds
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.64
Detected doublet rate = 0.3%
Estimated detectable doublet fraction = 13.4%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 2.1%
Elapsed time: 2.9 seconds
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.51
Detected doublet rate = 0.9%
Estimated detectable doublet fraction = 32.1%
Overall doublet rate:
	Expected   = 10.0%
	Estimated  = 2.9%
Elapsed time: 2.0 seconds


In [8]:
barcodes = [item for sublist in bcs for item in sublist]
doublets = [item for sublist in dbls for item in sublist]
scores = [item for sublist in scrs for item in sublist]
dbl_df = pd.DataFrame(zip(doublets, scores), columns = ['doublet', 'dbl_score'], index=barcodes)

In [9]:
adata_merge.obs['doublet'] = dbl_df['doublet']
adata_merge.obs['dbl_score'] = dbl_df['dbl_score']

print(sum(adata_merge.obs['doublet']))
print(adata_merge.n_obs)

4988
356944


In [10]:
adata_merge = adata_merge[adata_merge.obs['doublet'] == False].copy()

In [11]:
adata_merge

AnnData object with n_obs × n_vars = 351956 × 18093
    obs: 'dataset', 'sample', 'donor', 'timepoint', 'celltype', 'batch', 'size_factors', 'doublet', 'dbl_score'
    var: 'n_cells'
    uns: 'log1p'
    layers: 'counts'

In [12]:
adata_merge.obs['dataset'].value_counts()

nuig_2    110851
petti     101571
oetjen     71843
vanG       38093
granja     20353
nuig_1      9245
Name: dataset, dtype: int64

In [13]:
adata_merge.obs['celltype'].value_counts()

CMP.LMPP       40725
HSC            35091
CD14.Mono.2    34807
Late.Eryth     29803
CD4.M          26039
GMP            21522
CD4.N1         20898
CD14.Mono.1    17029
GMP.Neut       16959
CD8.CM         16328
Early.Eryth    15404
cDC            12530
NK             11143
CD8.N           9525
B               9223
CD4.N2          7485
CD16.Mono       6553
CD8.EM          5617
pDC             3757
CLP.1           3536
Early.Baso      2434
Pre.B           2430
Plasma          2394
CLP.2            724
Name: celltype, dtype: int64

In [14]:
adata_merge.obs['timepoint'].value_counts()

Healthy      110416
Diagnosis     83351
R             51023
MRD           36234
DG            32839
D0            15522
nan            7669
D113           2732
D29            1977
D31            1791
D18            1661
D171           1401
D14            1354
D15            1202
D20             945
D35             918
D41             385
D37             224
D34             202
D97              57
D49              53
Name: timepoint, dtype: int64

In [15]:
adata_merge.write('../data/bone_marrow_merge.h5ad')